In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")


from pyspark.sql.functions import *
from pyspark.sql.types import *



import folium
import html

In [2]:
data_path = '/home/osboxes/yelp-data/dataset/'
model_path = '/home/osboxes/yelp-data/model/'

### Load Data

##### Load business data

In [3]:
# load the business dataframe

business_df = spark.read.parquet(data_path + 'business-small.parquet')

In [4]:
# check the dataframe schema

business_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- business_name: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [5]:
# display 10 records

business_df.show(10)

+--------------------+--------------------+--------------------+--------------------+-------+-----+-----------+----------+-----------+-----+------------+--------------------+
|         business_id|       business_name|        neighborhood|             address|   city|state|postal_code|  latitude|  longitude|stars|review_count|          categories|
+--------------------+--------------------+--------------------+--------------------+-------+-----+-----------+----------+-----------+-----+------------+--------------------+
|qim0lD112TkDhm8Zy...|McCarthy's Irish Pub|         Upper Beach|1801 Gerrard Stre...|Toronto|   ON|    M4L 2B5|43.6780488|-79.3147736|  4.0|           5|[Pubs, Restaurant...|
|Wf5C8Amv_SlhoYE3_...|         Oishi Sushi|                    |    1325 Finch Ave W|Toronto|   ON|    M3J 2G5|43.7635097|-79.4907499|  2.0|          27|[Asian Fusion, Re...|
|Z1r6b30Tg0n0ME4-Z...|     Boardwalk Place|                    |1675 Lake Shore B...|Toronto|   ON|    M4W 3L6|43.6630096|-79

In [6]:
# businesses count

business_df.count()

6750

In [7]:
# create a temporary view to be used in SQL queries

business_df.createOrReplaceTempView("businesses")

##### Load user data

In [8]:
# Load the user dataframe
user_df = spark.read.parquet(data_path + 'user-small.parquet')

# check the user dataframe schema
user_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- yelping_since: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- fans: long (nullable = true)
 |-- average_stars: double (nullable = true)



In [9]:
# get users count

user_df.count()

66424

In [22]:
# display 10 records from the users dataframe

user_df.toPandas().head()

,user_id,user_name,review_count,yelping_since,useful,funny,cool,fans,average_stars
0,om5ZiponkpRqUNa3pVPiRg,Andrea,2559,2006-01-18,83681,10882,40110,835,3.94
1,Wc5L6iuvSNF5WGBlqIO8nw,Risa,1122,2011-07-30,26395,4880,19108,435,4.10
2,uxKSnOVAoEj4I6X9YhLBlg,Vivian,73,2013-03-02,34,5,2,8,3.54
3,s8bVHRqx6cI8F8HGf3A_og,Colleen,32,2014-12-18,19,3,7,2,4.15
4,xEajChTkzWIYTMLkYNoIIw,Di,71,2012-09-26,31,8,4,3,3.29


In [11]:
# create a user view for SQL queries

user_df.createOrReplaceTempView("users")

##### Load users' friends data

In [12]:

# Load the friend dataframe
friend_df = spark.read.parquet(data_path + 'friend-small.parquet')

# check the user dataframe schema
friend_df.printSchema()


root
 |-- user_id: string (nullable = true)
 |-- friend_id: string (nullable = true)



In [13]:
# get friends count

friend_df.count()

2047943

In [14]:
# display 10 records from the friend dataframe

friend_df.show(10)

+--------------------+--------------------+
|             user_id|           friend_id|
+--------------------+--------------------+
|om5ZiponkpRqUNa3p...|eoSSJzdprj3jxXyi9...|
|om5ZiponkpRqUNa3p...|QF0urZa-0bxga17Ze...|
|om5ZiponkpRqUNa3p...|U_sn0B-HWdTSlHNXI...|
|om5ZiponkpRqUNa3p...|1_4Q1prE_QcejmEH5...|
|om5ZiponkpRqUNa3p...|DhBu8qqXHqVVZGx73...|
|om5ZiponkpRqUNa3p...|n7luzOk8i8K5agYsF...|
|om5ZiponkpRqUNa3p...|pnfVIB7UhvCQ7X2K0...|
|om5ZiponkpRqUNa3p...|LWHrjeYiGbEiQJKc8...|
|om5ZiponkpRqUNa3p...|HsTEfv2QiJBI1Vcln...|
|om5ZiponkpRqUNa3p...|zrdkuuPFlNkLTSa9g...|
+--------------------+--------------------+
only showing top 10 rows



In [21]:
# create a friend view for SQL queries

friend_df.createOrReplaceTempView("friends")

##### Load review data

In [15]:
# create the review dataframe

review_df = spark.read.parquet(data_path + 'review-small.parquet')

# print the schema of the review dataframe
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)



In [17]:
# change the review_date to DateType

review_df = review_df.withColumn("review_date", review_df["review_date"].cast(DateType()))

review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- review_date: date (nullable = true)
 |-- review_text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)



In [18]:
# get the reviews count

review_df.count()

276887

In [19]:
# display 10 records from the review dataframe

review_df.show(10)

+--------------------+--------------------+--------------------+-----+-----------+--------------------+------+-----+----+
|           review_id|             user_id|         business_id|stars|review_date|         review_text|useful|funny|cool|
+--------------------+--------------------+--------------------+-----+-----------+--------------------+------+-----+----+
|Z5l99h18E3_g1GLcD...|djpMXOA1ic5wv3FPt...|mr4FiPaXTWlJ3qGzp...|    3| 2009-07-21|I left Table 17 f...|     3|    0|   0|
|Z3Fw292i0Eg8liW0D...|-pXs08gJq9ExIk275...|mr4FiPaXTWlJ3qGzp...|    3| 2008-12-13|for the time bein...|     1|    0|   0|
|hsKINx1dIKeFTDe-Z...|PTj29rhujYETuFlAZ...|mr4FiPaXTWlJ3qGzp...|    5| 2013-10-12|Love this place. ...|     1|    0|   1|
|oviMS8F4ACflGysxs...|3hLMY2dBEP1kYbd_y...|mr4FiPaXTWlJ3qGzp...|    5| 2013-02-17|Had a lovely even...|     0|    0|   0|
|SmH05C7YViVmVjaL4...|zZy5Jljx7rEvISiJ2...|mr4FiPaXTWlJ3qGzp...|    3| 2011-08-14|Table 17 is a cla...|     1|    0|   0|
|wNe5wh4wXjdRfkb6a...|B7

In [20]:
# create a review view to be used in SQL queries

review_df.createOrReplaceTempView("reviews")

### Friends Recommendations

From the reviews, get top 100 restaurants that the user did not visit (reviewd), and that rated as 4 or 5 by his friends; and then from those 100 select random sample of 10.

In [54]:
u_id = 'uxKSnOVAoEj4I6X9YhLBlg'

query = """
select business_id, count(*) as 4_5_stars_count 
from reviews
where user_id in
    (select f.friend_id from friends f
    inner join users u on f.friend_id = u.user_id
    where f.user_id = "{}") 
and stars >= 4 
and business_id not in (select business_id from reviews where user_id = "{}")
group by business_id
order by count(*) desc limit 100
""".format(u_id, u_id)

friend_recoms_df = sqlContext.sql(query)

friend_recoms_df.toPandas()

,business_id,4_5_stars_count
0,SGP1jf6k7spXkgwBlhiUVw,5
1,kOFDVcnj-8fd3doIpCQ06A,5
2,0a2O150ytxrDjDzXNfRWkA,4
3,k6zmSLmYAquCpJGKNnTgSQ,4
4,SjgeuBlgKER9yegpoxT99w,4
5,b4LmLgVdbhM-nc1IZc5Weg,3
6,-J6FVdY9pSgAdFmmalO-pQ,3
7,28adZ4lsuUeVB2aWzohK9g,3
8,G6EkDTXZ6zMUovg7JTG4YQ,3
9,RwRNR4z3kY-4OsFqigY5sw,3


In [58]:
friend_recoms_df = friend_recoms_df.sample(False, 0.5).limit(10)
friend_recoms_df.toPandas()

,business_id,4_5_stars_count
0,SGP1jf6k7spXkgwBlhiUVw,5
1,kOFDVcnj-8fd3doIpCQ06A,5
2,SjgeuBlgKER9yegpoxT99w,4
3,GcxE5hK_TaHP4EZFDYz2mg,3
4,crstB-H5rOfbXhV8pX0e6g,3
5,28adZ4lsuUeVB2aWzohK9g,3
6,HUYEadSbGSQNHXFmT2Ujjw,3
7,RwRNR4z3kY-4OsFqigY5sw,3
8,b4LmLgVdbhM-nc1IZc5Weg,3
9,a8pmtlVKf7NiSLI-4KejIw,2


In [38]:
def getBusinessDetails(in_bus):
    
    a = in_bus.alias("a")
    b = business_df.alias("b")
    
    return a.join(b, col("a.business_id") == col("b.business_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.business_name'),col('b.categories'),
                                                           col('b.stars'),col('b.review_count'),
                                                           col('b.latitude'),col('b.longitude')])
    

In [51]:
def showInMap(df):
    
    mp = folium.Map(location=[43.70011, -79.4163], zoom_start=12)

    for i, r in df.toPandas().iterrows():
        folium.Marker(
                    location =[r.latitude, r.longitude], 
                    popup = html.escape(r["business_name"]) + '<br>' + 'Stars: ' + str(r.stars) + '<br>' + 'Reviews: ' + str(r.review_count),    
                    icon = folium.Icon(color='orange')).add_to(mp)
    return mp

In [55]:
def getFriendRecoms(u_id, sim_bus_limit=10):
    
    query = """
    select business_id, count(*) as 4_5_stars_count 
    from reviews
    where user_id in
        (select f.friend_id from friends f
        inner join users u on f.friend_id = u.user_id
        where f.user_id = "{}") 
    and stars >= 4 
    and business_id not in (select business_id from reviews where user_id = "{}")
    group by business_id
    order by count(*) desc limit 100
    """.format(u_id, u_id)

    friend_recoms_df = sqlContext.sql(query)
    
    friend_recoms_df = friend_recoms_df.sample(False, 0.5).limit(sim_bus_limit)

    return getBusinessDetails(friend_recoms_df)

In [56]:
# test recoms for a user

u_id = 'uxKSnOVAoEj4I6X9YhLBlg'

friend_recom_df = getFriendRecoms(u_id)

print("Businesses recommended to user by best useful friends:")
friend_recom_df.toPandas()

Businesses recommended to user by best useful friends:


,business_id,4_5_stars_count,business_name,categories,stars,review_count,latitude,longitude
0,SGP1jf6k7spXkgwBlhiUVw,5,Kekou Gelato House,"[Food, Restaurants, Ice Cream & Frozen Yogurt,...",4.5,332,43.655983,-79.392686
1,kOFDVcnj-8fd3doIpCQ06A,5,Mildred's Temple Kitchen,"[Comfort Food, Event Planning & Services, Vege...",4.0,472,43.639911,-79.420424
2,0a2O150ytxrDjDzXNfRWkA,4,Miku Toronto,"[Sushi Bars, Restaurants, Seafood, Japanese]",4.0,384,43.641235,-79.377370
3,G6EkDTXZ6zMUovg7JTG4YQ,3,Vietnam Noodle Star,"[Restaurants, Vietnamese, Noodles]",3.5,148,43.804603,-79.287842
4,RwRNR4z3kY-4OsFqigY5sw,3,Uncle Tetsu's Japanese Cheesecake,"[Desserts, Japanese, Restaurants, Bakeries, Food]",3.5,806,43.655969,-79.384013
5,Yv4P4qUwd7F-qQ4Y4eD1JQ,3,Han Ba Tang,"[Nightlife, Pubs, Lounges, Korean, Asian Fusio...",3.5,213,43.762928,-79.411511
6,dTuT_G3Zp79RZmnF3oxfiA,3,The Bier Markt,"[Belgian, Nightlife, Bars, Gastropubs, Canadia...",3.0,197,43.647095,-79.373915
7,MhiBpIBNTCAm1Xd3WzRzjQ,3,Messini Authentic Gyros,"[Mediterranean, Sandwiches, Greek, Restaurants...",3.5,372,43.677691,-79.350536
8,9_CGhHMz8698M9-PkVf0CQ,2,Little Coxwell Vietnamese & Thai Cuisine,"[Vietnamese, Thai, Restaurants]",4.0,109,43.696175,-79.329092
9,ofw8aDSEg1HoQdmCgvLtaQ,2,The Pie Commission,"[Canadian (New), Fast Food, Food, Do-It-Yourse...",4.5,183,43.623881,-79.512074


In [57]:
showInMap(friend_recom_df)